**TODO:**

- 3-5 CNN models
- From simple 1 layer fully connected to transfer learning
- May review ensemble
- Need to review books vs movies(tvshows) model

In [1]:
import keras
import keras_util
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

#### Input Data

In [3]:
notbook_path = '/home/jason/DeepLearning/github/goodscifi/development/notebooks/'
home_path = '/home/jason/DeepLearning/github/goodscifi/development/'
data_path = '/home/jason/DeepLearning/github/goodscifi/development/data/'

In [4]:
# Not using 'sample' since I have less than 6k total data
training_data = data_path+'dataset/movies/train/'
validation_data = data_path+'dataset/movies/valid/'

In [5]:
# Create numpy array of data
train_data_array = keras_util.get_data(training_data)
val_data_array = keras_util.get_data(validation_data)

Found 4348 images belonging to 2 classes.
Found 1448 images belonging to 2 classes.


In [6]:
# Save and use for datagen.fit
keras_util.save_array(data_path+'dataset/movies/train_data.bcolz',train_data_array)
keras_util.save_array(data_path+'dataset/movies/val_data.bcolz',val_data_array)

In [7]:
train_batches = keras_util.get_batches(training_data) # update target_size
val_batches = keras_util.get_batches(validation_data)

Found 4348 images belonging to 2 classes.
Found 1448 images belonging to 2 classes.


In [8]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True)

In [9]:
image_datagen = ImageDataGenerator(**data_gen_args)

In [10]:
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_data_array, augment=True, seed=seed)

In [46]:
# Get train_batches with datagen augmentation (normalized)
# update target_size and batch_size
train_batches_aug = keras_util.get_batches(training_data,gen=image_datagen, batch_size=16) 
val_batches_aug = keras_util.get_batches(validation_data,gen=image_datagen, batch_size=16)

Found 4348 images belonging to 2 classes.
Found 1448 images belonging to 2 classes.


In [41]:
train_batches_aug.image_shape

(224, 224, 3)

#### Fully Connected NN

In [48]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(224,224,3))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(32, activation='relu')(inputs)
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
dense_25 (Dense)             (None, 224, 224, 32)      128       
_________________________________________________________________
flatten_3 (Flatten)          (None, 1605632)           0         
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 3211266   
Total params: 3,211,394
Trainable params: 3,211,394
Non-trainable params: 0
_________________________________________________________________


In [52]:
print(train_batches_aug.samples, train_batches_aug.batch_size)

4348 16


In [53]:
history = model.fit_generator(train_batches_aug, 
                    steps_per_epoch=train_batches_aug.samples/train_batches_aug.batch_size,
                    epochs=5)

Epoch 1/5
272/271 [==============================] - 29s - loss: 2.2283 - acc: 0.8618     

In [55]:
keras_util.plot_train(history)

KeyError: 'val_acc'

#### Test Model Across Learning Rates

In [ ]:
# Import the SGD optimizer
from keras.optimizers import SGD

# Create list of learning rates: lr_to_test
lr_to_test = [0.000001, 0.01, 1]

# Loop over learning rates
for lr in lr_to_test:
    print('\n\nTesting model with learning rate: %f\n'%lr )
    
    # Build new model to test, unaffected by previous models
    model = get_new_model()
    
    # Create SGD optimizer with specified learning rate: my_optimizer
    # Use SGD rather than Adam to help narrow down best learning rate to start with
    my_optimizer = SGD(lr=lr)
    
    # Compile the model
    model.compile(optimizers=my_optimizer, loss='categorical_crossentroy')
    
    # Fit the model
    model.fit(predictors, target)